In [1]:
import sys
sys.path.append('..\\helpers')

import cv2
import numpy as np
from tqdm import tqdm
import os
import csv
import uuid

from preprocessing import Preprocessor
from segmentation import Segmentation
from feature_extractor import FeatureExtractor

fe = FeatureExtractor()
preprocessor = Preprocessor()
seg = Segmentation()

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

The code below takes two input images, ground truth and segmentation scale result. It will show the iou per mass on top of the segmentation image and a bounding box.

The task is to integrate this approach with feature extraction, so the segmentation result will segment the raw image, then the iou of that segmentation will be calculated as below. Then based on the iou, two csv files will be created (tp, fn), each of them contain the extracted features.

### 1. Separating Segmented Masses (IOU calculation)

In [2]:
def upsample(image):
    height, width = image.shape[:2]
    new_height = int(height * 4)
    new_width = int(width * 4)
    
    return cv2.resize(image.copy(), (new_width, new_height), interpolation=cv2.INTER_CUBIC)

def binarize(image):
    threshold = 128
    _, binarized_image = cv2.threshold(image, threshold, 255, cv2.THRESH_BINARY)
    return binarized_image

def show_image(labe, image):    
    cv2.imshow(labe, cv2.resize(image, (image.shape[1]//3, image.shape[0]//3)))
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def convert_to_8bit(image_16bit):
    if not isinstance(image_16bit, np.ndarray) or image_16bit.dtype != np.uint16:
        raise ValueError("Input must be a 16-bit numpy array")

    # Scale the 16-bit image to the range of 0-255
    image_8bit = (image_16bit / np.max(image_16bit) * 255).astype(np.uint8)

    return image_8bit


In [3]:
# Load binary image
image_gt = cv2.imread('../dataset/processed/groundtruth/22580341_5eae9beae14d26fd_MG_R_CC_ANON.tif', 0)  # Replace with your ground truth image file
image_segmentation = cv2.imread('../dataset/output/segmentation/22580341_5eae9beae14d26fd_MG_R_CC_ANON_7.tif', 0)  # Replace with your segmentation image file

# upsample gt
image_gt = upsample(image_gt)

# upsample seg
image_segmentation = upsample(image_segmentation)



# Threshold the image to convert it into binary format (if necessary)
_, binary_image_gt = cv2.threshold(image_gt, 127, 255, cv2.THRESH_BINARY)
_, binary_image_segmentation = cv2.threshold(image_segmentation, 127, 255, cv2.THRESH_BINARY)

# Find contours of the masses
contours_gt, _ = cv2.findContours(binary_image_gt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_segmentation, _ = cv2.findContours(binary_image_segmentation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

output_image = image_segmentation

# iterate over gt contours
for i, contour_gt in enumerate(contours_gt):

    gt_candidate = np.zeros_like(binary_image_gt)
    cv2.drawContours(gt_candidate, [contour_gt], 0, (255, 255, 255), cv2.FILLED)

    max_iou = 0
    
    # iterate over segmentation contours
    for j, contour_segmentation in enumerate(contours_segmentation):
        mask_candidate = np.zeros_like(binary_image_segmentation)
        cv2.drawContours(mask_candidate, [contour_segmentation], 0, (255, 255, 255), cv2.FILLED)
        
        # Display the output image
        # cv2.imshow(f'mask_candidate {i, j}', cv2.resize(mask_candidate, (800, 1000)))
        # cv2.imshow(f'binary_image_gt {i, j}', cv2.resize(binary_image_gt, (800, 1000)))
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        
        # Compute the intersection and union images
        intersection = cv2.bitwise_and(mask_candidate, gt_candidate)
        union = cv2.bitwise_or(mask_candidate, gt_candidate)

        # Compute the number of white pixels in the intersection and union images
        intersection_pixels = cv2.countNonZero(intersection)
        union_pixels = cv2.countNonZero(union)

        # Compute the IoU
        iou = intersection_pixels / union_pixels
        
        if iou > max_iou:
            max_iou = iou
            print("max_iou: ", max_iou)
            
            # Get the bounding rectangle of the contour
            # let the size be as the contours_segmentation not contours_gt so that the bb is surrounding the full seg mass
            x, y, w, h = cv2.boundingRect(contour_segmentation) 
            
            # Draw the bounding box with a random color
            cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 255, 255), 3)
    

    
    # Optionally, display the index of each bounding box
    cv2.putText(output_image, "IOU: " + str(max_iou), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4)
cv2.putText(output_image, "Number of masses: " + str(len(contours_segmentation)), (50, output_image.shape[0] - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4)
    
# Display the output image
cv2.imshow('IOU per mass on segmented scale.', cv2.resize(output_image, (output_image.shape[1]//3, output_image.shape[0]//3)))
cv2.imshow('Ground Truth image', cv2.resize(image_gt, (image_gt.shape[1]//3, image_gt.shape[0]//3)))
cv2.waitKey(0)
cv2.destroyAllWindows()

max_iou:  0.6004385448551919
max_iou:  0.37717680368281753


### 2. Preprocessing Raw Images (Cropping Only)

This step is to make sure that the raw positive image and the upscaled segmented scale are the same size

In [4]:
raw = cv2.imread('../dataset/positive_images/20586908_6c613a14b80a8591_MG_R_CC_ANON.tif', cv2.IMREAD_UNCHANGED)

cropped_raw = preprocessor.crop_raw(raw)
upsampled_raw = upsample(cropped_raw)

print(upsampled_raw.shape, upsampled_raw.dtype)

cv2.imshow('cropped_raw', cv2.resize(upsampled_raw, (upsampled_raw.shape[1]//3, upsampled_raw.shape[0]//3)))
cv2.waitKey(0)
cv2.destroyAllWindows()

(3456, 1952) uint16


The code below preprocess the raw images. Run it only once. 

In [5]:
# positive_images_dir = r"../dataset/images/" ## crop all not only positive images
# cropped_raw_dir = r"../dataset/processed/cropped_raw/"


# if not os.path.isdir(cropped_raw_dir):
#     os.makedirs(cropped_raw_dir)
#     print(f"New directory created '{cropped_raw_dir}'")
    

# for idx, file_name in tqdm(enumerate(os.listdir(positive_images_dir))):
#     # read raw image 
#     input_image = cv2.imread(
#         os.path.join(
#             positive_images_dir, 
#             file_name
#         ), cv2.IMREAD_UNCHANGED)
    
#     # create file dir path
#     file_dir = os.path.join(cropped_raw_dir, file_name)
    
#     # crop raw image
#     cropped_raw = preprocessor.crop_raw(input_image)
    
#     # upsample, they were downsampled in the cropping to handle +- dimension approximation
#     upsampled_raw = upsample(cropped_raw)
    
#     # export
#     cv2.imwrite(file_dir, upsampled_raw.astype(np.uint16))

### 3. Exporting all segmentation scales, last updated 

The code below export all scales segmentations. Run it only once.

In [6]:
# positive_images_dir = r"..\\dataset\\positive_images"
# segmentation_output_dir = r"..\\dataset\\output\\segmentation"

# for idx, file_name in tqdm(enumerate(os.listdir(positive_images_dir))):
#     input_image = cv2.imread(
#         os.path.join(
#             os.path.dirname("../dataset/processed/positive_images/"), 
#             file_name
#         ), cv2.IMREAD_UNCHANGED)

#     _, _, _, masses = seg.pipeline(input_image)
    
#     for index, img in enumerate(masses):
#         file_dir = os.path.join(segmentation_output_dir, file_name.split('.tif')[0] + '_' + str(index) + '.tif')
#         cv2.imwrite(file_dir, masses[index].astype(np.uint8))

In [7]:
# negative_images_dir = r"..\\dataset\\negative_images"
# segmentation_output_dir = r"..\\dataset\\output\\segmentation_negatives"

# if not os.path.isdir(segmentation_output_dir):
#     os.makedirs(segmentation_output_dir)
#     print(f"New directory created '{segmentation_output_dir}'")

# for idx, file_name in tqdm(enumerate(os.listdir(negative_images_dir))):
#     input_image = cv2.imread(
#         os.path.join(
#             os.path.dirname("../dataset/processed/negative_images/"), 
#             file_name
#         ), cv2.IMREAD_UNCHANGED)

#     _, _, _, masses = seg.pipeline(input_image)
    
#     for index, img in enumerate(masses):
#         file_dir = os.path.join(segmentation_output_dir, file_name.split('.tif')[0] + '_' + str(index) + '.tif')
#         cv2.imwrite(file_dir, masses[index].astype(np.uint8))

### 4. Exploring masking raw images with the segmented scale result

In [8]:
raw_image = cv2.imread('../dataset/processed/cropped_raw/20586908_6c613a14b80a8591_MG_R_CC_ANON.tif', cv2.IMREAD_GRAYSCALE)
binary_mask = cv2.imread('../dataset/output/segmentation/20586908_6c613a14b80a8591_MG_R_CC_ANON_7.tif', cv2.IMREAD_GRAYSCALE)

binary_mask = upsample(binary_mask)

print(raw_image.dtype)

# Apply binary mask to the raw image
segmented_image = cv2.bitwise_and(raw_image, binary_mask)

print(segmented_image.dtype)

# Display the final result
cv2.imshow('Segmented Image', cv2.resize(segmented_image, (raw_image.shape[1]//3, raw_image.shape[0]//3)))
cv2.imshow('binary_mask Image', cv2.resize(binary_mask, (binary_mask.shape[1]//3, binary_mask.shape[0]//3)))

cv2.waitKey(0)
cv2.destroyAllWindows()

uint8
uint8


The code below segment the raw images. Run it only once.

In [9]:
# cropped_raw_dir = r"../dataset/processed/cropped_raw/"
# segmentation_output_regions_dir = r"..\\dataset\\output\\segmentation-regions"
# positive_images_dir = r"../dataset/positive_images/"

# if not os.path.isdir(segmentation_output_regions_dir):
#     os.makedirs(segmentation_output_regions_dir)
#     print(f"New directory created '{segmentation_output_regions_dir}'")

# for idx, file_name in tqdm(enumerate(os.listdir(positive_images_dir))):
#     input_image = cv2.imread(
#         os.path.join(
#             os.path.dirname("../dataset/processed/positive_images/"), 
#             file_name
#         ), cv2.IMREAD_UNCHANGED)

#     _, _, _, masses = seg.pipeline(input_image)
    
#     for index, img in enumerate(masses):
#         file_dir = os.path.join(segmentation_output_regions_dir, file_name.split('.tif')[0] + '_' + str(index) + '.tif')
          
#         # upsample scale 
#         binary_mask = upsample(img)
        
#         # read the cropped raw image 
#         raw_image = cv2.imread(
#             os.path.join(cropped_raw_dir, file_name),
#             cv2.IMREAD_GRAYSCALE)
        
# #         Apply binary mask to the raw image
#         segmented_image = cv2.bitwise_and(raw_image, binary_mask)
#         cv2.imwrite(file_dir, segmented_image)


### 5. Now we need to integrate everything togather (mask and segmentation iteration, IOU calculation, feature extraction)

We can try exporting two CSV files (positives and negatives). An IOU threshold to be set to determine which segmentation should be exported to either CSV files. 

In [10]:
# # trying feature extraction on a single image
# segmentation_image  = r"../dataset/output/segmentation-regions/20586908_6c613a14b80a8591_MG_R_CC_ANON_2.tif"

# segmentation_image = cv2.imread(segmentation_image, cv2.IMREAD_UNCHANGED)

# mask = np.zeros_like(segmentation_image, dtype=np.uint8)



# contours_segmentation, _ = cv2.findContours(segmentation_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


# # Extract texture features for each region
# texture_features = []


# # for region_label in np.unique(segmentation_image):
# #     region_mask = np.zeros_like(segmentation_image)
# #     region_mask[segmentation_image == region_label] = 255
    

# cv2.imshow('Segmented Image', cv2.resize(mask, (mask.shape[1]//3, mask.shape[0]//3)))
# cv2.waitKey(0)
# cv2.destroyAllWindows()



In [16]:
## Create a new CSV file

file_id = str(uuid.uuid4()) 
csv_filename = 'output-'+file_id+'.csv'

print("New csv file created: ", csv_filename)

New csv file created:  output-748343c6-bedf-41c0-9696-0ab3a3085ba3.csv


Extracting features from positive images

In [17]:
groundtruth_path = r'../dataset/processed/groundtruth'
segmentation_output_dir = r"../dataset/output/segmentation"
cropped_raw_dir = r"../dataset/processed/cropped_raw/"
features_dir = r"../dataset/output/features"

if not os.path.isdir(features_dir):
    os.makedirs(features_dir)
    print(f"New directory created '{features_dir}'")


with open(os.path.join(features_dir, csv_filename), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # let everything be inside this scope to be able to access the csv later
    IOU_dataset_vector = []
    # loop over all of the gt masks
    for idx, file_name in tqdm(enumerate(os.listdir(groundtruth_path))):
        # print(file_name)

        # getting the ground truth image
        ground_truth = cv2.imread(
            os.path.join(groundtruth_path, file_name)
            , cv2.IMREAD_UNCHANGED)

        # upsample
        ground_truth = upsample(ground_truth)

        # binarize
        ground_truth = binarize(ground_truth)

        # find contours
        contours_gt, _ = cv2.findContours(ground_truth, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # print(len(contours_gt), file_name)

        # getting the cropped raw image
        cropped_raw = cv2.imread(
            os.path.join(cropped_raw_dir, file_name)
            , cv2.IMREAD_UNCHANGED)

        cropped_raw = convert_to_8bit(cropped_raw)

        # print(cropped_raw.shape, cropped_raw.dtype)
        # print(ground_truth.shape, ground_truth.dtype)

        # show_image("cropped_raw", cropped_raw)
        # show_image("ground_truth", ground_truth)

        IOU_image_vector = []
        # iterate over all gt contours then over all the scales then on every contours for a single scale segmentation image
        for i, contour_gt in enumerate(contours_gt):

            gt_candidate = np.zeros_like(ground_truth)
            cv2.drawContours(gt_candidate, [contour_gt], 0, (255, 255, 255), cv2.FILLED)

            #show_image("gt_candidate", gt_candidate)

            max_iou = 0

            # loop over all of the segmentation masks
            # image names has an extra number at the end indicating the scale number
            for index in range(10):

                segmentation_mask_file_dir = os.path.join(segmentation_output_dir, file_name.split('.tif')[0] + '_' + str(index) + '.tif')
                # print(file_dir)

                segmentation_mask = cv2.imread(segmentation_mask_file_dir, cv2.IMREAD_UNCHANGED)
                segmentation_mask = upsample(segmentation_mask)

                # print(segmentation_mask.shape, segmentation_mask.dtype)
                # show_image(f"{index}", segmentation_mask)

                # binarize
                segmentation_mask = binarize(segmentation_mask)


                # find contours
                contours_segmentation, _ = cv2.findContours(segmentation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                # print("contours_segmentation: ", len(contours_segmentation))

                # iterate over segmentation contours
                for j, contour_segmentation in enumerate(contours_segmentation):
                    mask_candidate = np.zeros_like(segmentation_mask)
                    cv2.drawContours(mask_candidate, [contour_segmentation], 0, (255, 255, 255), cv2.FILLED)                

                    #### IOU Calculation code starts ####
                    # Compute the intersection and union images
                    intersection = cv2.bitwise_and(mask_candidate, gt_candidate)
                    union = cv2.bitwise_or(mask_candidate, gt_candidate)

                    # Compute the number of white pixels in the intersection and union images
                    intersection_pixels = cv2.countNonZero(intersection)
                    union_pixels = cv2.countNonZero(union)

                    # Compute the IoU
                    iou = intersection_pixels / union_pixels

                    if iou > max_iou:
                        max_iou = iou
                        # print("max_iou: ", max_iou)

                        # Get the bounding rectangle of the contour
                        # let the size be as the contours_segmentation not contours_gt so that the bb is surrounding the full seg mass
                        # x, y, w, h = cv2.boundingRect(contour_segmentation) 

                        # Draw the bounding box with a random color
                        # cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 255, 255), 3)

                    # show the image region with its iou as a file name
                    # show_image(f"mask_candidate, IOU:{iou}", mask_candidate)

                    #### Bitwise to get exact region from cropped raw ####
                    # print(cropped_raw.dtype, mask_candidate.dtype)

                    # For glcm features, we extract them from the bounding rectange of the contour
                    output_image = mask_candidate

                    # Get the bounding rectangle of the contour
                    x, y, w, h = cv2.boundingRect(contour_segmentation) 

                    # Draw the bounding box with a random color (just for display)
                    # cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 255, 255), 3)
                    # show_image(f"segmented_region_raw, IOU:{iou}", output_image)

                    cropped_raw_box = cropped_raw[y:y+h, x:x+w]
                    # show_image(f"cropped_raw_box, IOU:{iou}", cropped_raw_box)

                    ### Extracting texture features using glcm
                    texture_features = fe.extract_texture_features(cropped_raw_box)
                    # print(texture_features)

                    ### Extracting shape features
                    shape_features = fe.extract_shape_features(image=cropped_raw, labels=mask_candidate)
                    # print(shape_features)

                    ### Extracting intensity features
                    intensity_features = fe.extract_intensity_features(image=cropped_raw, labels=mask_candidate)
                    # print(intensity_features)

                    ### Concatinate all features with labels
                    combined_features = np.concatenate([texture_features, shape_features, intensity_features])
                    # print(combined_features.ndim)
                    
                    # Reshape to a 2D array
                    combined_features = combined_features.reshape(-1, 2)
                    # print(combined_features.ndim)

                    # Calculate the minimum and maximum values of combined_features
                    # Note -> Norm every col separately
                    min_value = np.min(combined_features, axis=0)
                    max_value = np.max(combined_features, axis=0)

                    # Perform min-max scaling
                    normalized_features = (combined_features - min_value) / (max_value - min_value)
                    normalized_features = normalized_features.reshape(-1, 1)
                    # print(normalized_features)

                    # Append the label to the feature vector
                    candidate_label = 1 if iou > 0.5 else 0
                    normalized_features = np.append(normalized_features, candidate_label)
                    # print(normalized_features.ndim)

                    # Write all of the features with the labels in a single file
                    writer.writerow(normalized_features)



                    # segmented_region_raw = cv2.bitwise_and(cropped_raw, mask_candidate)
                    # show_image(f"segmented_region_raw, IOU:{iou}", segmented_region_raw)

                    #### feature extraction ####
                    # extract features from every scale
                    # features_tp = [
                    #     [features_scale_1]
                    # ]

                    # Note that we iterate twice, and calculate iou for each gt mass. Should we push outside this loop into the feature vector? No since IOU could increase on the second loop of gt
                    # print(segmented_region_raw.shape, segmented_region_raw.dtype)
                    # feature_region = fe.extract_texture_features(segmented_region_raw)
                    # print(j, feature_region)

                    # break

                # break



                    # Optionally, display the index of each bounding box
                    # cv2.putText(output_image, "IOU: " + str(max_iou), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4)
                # cv2.putText(output_image, "Number of masses: " + str(len(contours_segmentation)), (50, output_image.shape[0] - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4)
                # show_image("output_image", output_image)

            IOU_image_vector.append(max_iou)
            # break


        print(IOU_dataset_vector)
        IOU_dataset_vector += IOU_image_vector
        # break




1it [00:34, 34.97s/it]

[]


2it [00:54, 25.60s/it]

[0, 0.6548204440470868]


3it [01:53, 41.18s/it]

...

[0, 0.6548204440470868, 0.36697748464862406, 0.6933685688330994, 0.7680387588298384, 0.6127588627588627, 0.7252385865359814, 0.0003047597938715576, 0.7408958901698495, 0.6626438168798777, 0.24205674283319775, 0.6371020830735412, 0.7369217717334406, 6.248828344685372e-05, 0.2231300381178529, 0.24178662442316964, 0.6828319113905104, 0.6439461333169729, 0.04739088189906457, 0.46654842485008624, 0.7540482697738963, 0.8339769493596491, 0.1846355827792001, 0.8203670534485892, 0.7047907380932358, 0.7972200349207436, 0.5005495713343592, 0.8391121362349664, 0.7638112463005591, 0.7436311655737531, 0.7655913283337643, 0.013401452618335018, 0.7344119815310597, 0.690811736406507, 0.4259724668214454, 0.001355197181189863, 0.7735618006442281, 0.7415006262097233, 0.823390533570024, 0.9036922057310288, 0, 0.8596660297111306, 0.6924840910393855, 0.43135153598189696, 0.7888557231478747, 0.4825931183667951, 0.6083284404498441, 0.8195015968554231, 0.8574138242490926, 0.809812633363538, 0.7472780469087247, 

106it [18:16,  7.67s/it]

[0, 0.6548204440470868, 0.36697748464862406, 0.6933685688330994, 0.7680387588298384, 0.6127588627588627, 0.7252385865359814, 0.0003047597938715576, 0.7408958901698495, 0.6626438168798777, 0.24205674283319775, 0.6371020830735412, 0.7369217717334406, 6.248828344685372e-05, 0.2231300381178529, 0.24178662442316964, 0.6828319113905104, 0.6439461333169729, 0.04739088189906457, 0.46654842485008624, 0.7540482697738963, 0.8339769493596491, 0.1846355827792001, 0.8203670534485892, 0.7047907380932358, 0.7972200349207436, 0.5005495713343592, 0.8391121362349664, 0.7638112463005591, 0.7436311655737531, 0.7655913283337643, 0.013401452618335018, 0.7344119815310597, 0.690811736406507, 0.4259724668214454, 0.001355197181189863, 0.7735618006442281, 0.7415006262097233, 0.823390533570024, 0.9036922057310288, 0, 0.8596660297111306, 0.6924840910393855, 0.43135153598189696, 0.7888557231478747, 0.4825931183667951, 0.6083284404498441, 0.8195015968554231, 0.8574138242490926, 0.809812633363538, 0.7472780469087247, 

107it [18:18, 10.27s/it]

[0, 0.6548204440470868, 0.36697748464862406, 0.6933685688330994, 0.7680387588298384, 0.6127588627588627, 0.7252385865359814, 0.0003047597938715576, 0.7408958901698495, 0.6626438168798777, 0.24205674283319775, 0.6371020830735412, 0.7369217717334406, 6.248828344685372e-05, 0.2231300381178529, 0.24178662442316964, 0.6828319113905104, 0.6439461333169729, 0.04739088189906457, 0.46654842485008624, 0.7540482697738963, 0.8339769493596491, 0.1846355827792001, 0.8203670534485892, 0.7047907380932358, 0.7972200349207436, 0.5005495713343592, 0.8391121362349664, 0.7638112463005591, 0.7436311655737531, 0.7655913283337643, 0.013401452618335018, 0.7344119815310597, 0.690811736406507, 0.4259724668214454, 0.001355197181189863, 0.7735618006442281, 0.7415006262097233, 0.823390533570024, 0.9036922057310288, 0, 0.8596660297111306, 0.6924840910393855, 0.43135153598189696, 0.7888557231478747, 0.4825931183667951, 0.6083284404498441, 0.8195015968554231, 0.8574138242490926, 0.809812633363538, 0.7472780469087247, 

Extracting features from negative images

In [18]:
groundtruth_path = r'../dataset/processed/groundtruth'
segmentation_output_dir = r"../dataset/output/segmentation_negatives"
cropped_raw_dir = r"../dataset/processed/cropped_raw/"
features_dir = r"../dataset/output/features"
negative_images_dir = r"../dataset/negative_images"

if not os.path.isdir(features_dir):
    os.makedirs(features_dir)
    print(f"New directory created '{features_dir}'")

with open(os.path.join(features_dir, csv_filename), 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    # let everything be inside this scope to be able to access the csv later
    # IOU_dataset_vector = []
    # loop over all of the gt masks
    for idx, file_name in tqdm(enumerate(os.listdir(negative_images_dir))):
        # print(file_name)
        

        # getting the ground truth image
#         ground_truth = cv2.imread(
#             os.path.join(groundtruth_path, file_name)
#             , cv2.IMREAD_UNCHANGED)

        # upsample
#         ground_truth = upsample(ground_truth)

        # binarize
#         ground_truth = binarize(ground_truth)

        # find contours
#         contours_gt, _ = cv2.findContours(ground_truth, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # print(len(contours_gt), file_name)

        # getting the cropped raw image
        cropped_raw = cv2.imread(
            os.path.join(cropped_raw_dir, file_name)
            , cv2.IMREAD_UNCHANGED)

        # print(cropped_raw.shape, cropped_raw.dtype)

        cropped_raw = convert_to_8bit(cropped_raw)

        # print(cropped_raw.shape, cropped_raw.dtype)
#         print(ground_truth.shape, ground_truth.dtype)

        # show_image("cropped_raw", cropped_raw)
        # show_image("ground_truth", ground_truth)

        # IOU_image_vector = []
        # iterate over all gt contours then over all the scales then on every contours for a single scale segmentation image
        # # for i, contour_gt in enumerate(contours_gt):

            # gt_candidate = np.zeros_like(ground_truth)
            # cv2.drawContours(gt_candidate, [contour_gt], 0, (255, 255, 255), cv2.FILLED)

            #show_image("gt_candidate", gt_candidate)

            # max_iou = 0

            # loop over all of the segmentation masks
            # image names has an extra number at the end indicating the scale number
        for index in range(10):

            segmentation_mask_file_dir = os.path.join(segmentation_output_dir, file_name.split('.tif')[0] + '_' + str(index) + '.tif')
            # print(file_dir)

            segmentation_mask = cv2.imread(segmentation_mask_file_dir, cv2.IMREAD_UNCHANGED)
            segmentation_mask = upsample(segmentation_mask)

            # print(segmentation_mask.shape, segmentation_mask.dtype)
            # show_image(f"{index}", segmentation_mask)

            # binarize
            segmentation_mask = binarize(segmentation_mask)
            
            
            # find contours
            contours_segmentation, _ = cv2.findContours(segmentation_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # print("contours_segmentation: ", len(contours_segmentation))

            # iterate over segmentation contours
            for j, contour_segmentation in enumerate(contours_segmentation):
                mask_candidate = np.zeros_like(segmentation_mask)
                cv2.drawContours(mask_candidate, [contour_segmentation], 0, (255, 255, 255), cv2.FILLED)                

                #### IOU Calculation code starts ####
                # Compute the intersection and union images
                # intersection = cv2.bitwise_and(mask_candidate, gt_candidate)
                # union = cv2.bitwise_or(mask_candidate, gt_candidate)

                # Compute the number of white pixels in the intersection and union images
                # intersection_pixels = cv2.countNonZero(intersection)
                # union_pixels = cv2.countNonZero(union)

                # Compute the IoU
                # iou = intersection_pixels / union_pixels

#                 if iou > max_iou:
#                     max_iou = iou
                    # print("max_iou: ", max_iou)

                    # Get the bounding rectangle of the contour
                    # let the size be as the contours_segmentation not contours_gt so that the bb is surrounding the full seg mass
                    # x, y, w, h = cv2.boundingRect(contour_segmentation) 

                    # Draw the bounding box with a random color
                    # cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 255, 255), 3)

                # show the image region with its iou as a file name
                # show_image(f"mask_candidate, IOU:{iou}", mask_candidate)

                #### Bitwise to get exact region from cropped raw ####
                # print(cropped_raw.dtype, mask_candidate.dtype)

                # For glcm features, we extract them from the bounding rectange of the contour
                output_image = mask_candidate

                # Get the bounding rectangle of the contour
                x, y, w, h = cv2.boundingRect(contour_segmentation) 

                # Draw the bounding box with a random color (just for display)
                # cv2.rectangle(output_image, (x, y), (x + w, y + h), (255, 255, 255), 3)
                # how_image(f"segmented_region_raw, IOU:{0}", output_image)

                cropped_raw_box = cropped_raw[y:y+h, x:x+w]
                # show_image(f"cropped_raw_box, IOU:{iou}", cropped_raw_box)

                ### Extracting texture features using glcm
                texture_features = fe.extract_texture_features(cropped_raw_box)
                # print(texture_features)

                ### Extracting shape features
                shape_features = fe.extract_shape_features(image=cropped_raw, labels=mask_candidate)
                # print(shape_features)

                ### Extracting intensity features
                intensity_features = fe.extract_intensity_features(image=cropped_raw, labels=mask_candidate)
                # print(intensity_features)

                ### Concatinate all features with labels
                combined_features = np.concatenate([texture_features, shape_features, intensity_features])
                # print(combined_features.ndim)

                # Reshape to a 2D array
                combined_features = combined_features.reshape(-1, 2)
                # print(combined_features.ndim)

                # Calculate the minimum and maximum values of combined_features
                # Note -> Norm every col separately
                min_value = np.min(combined_features, axis=0)
                max_value = np.max(combined_features, axis=0)

                # Perform min-max scaling
                normalized_features = (combined_features - min_value) / (max_value - min_value)
                normalized_features = normalized_features.reshape(-1, 1)
                # print(normalized_features)

                # Append the label to the feature vector
                candidate_label = 0
                normalized_features = np.append(normalized_features, candidate_label)
                # print(normalized_features)

                # Write all of the features with the labels in a single file
                writer.writerow(normalized_features)



                # segmented_region_raw = cv2.bitwise_and(cropped_raw, mask_candidate)
                # show_image(f"segmented_region_raw, IOU:{iou}", segmented_region_raw)

                #### feature extraction ####
                # extract features from every scale
                # features_tp = [
                #     [features_scale_1]
                # ]

                # Note that we iterate twice, and calculate iou for each gt mass. Should we push outside this loop into the feature vector? No since IOU could increase on the second loop of gt
                # print(segmented_region_raw.shape, segmented_region_raw.dtype)
                # feature_region = fe.extract_texture_features(segmented_region_raw)
                # print(j, feature_region)

                # break

            # break



                # Optionally, display the index of each bounding box
                # cv2.putText(output_image, "IOU: " + str(max_iou), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4)
            # cv2.putText(output_image, "Number of masses: " + str(len(contours_segmentation)), (50, output_image.shape[0] - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 4)
            # show_image("output_image", output_image)

        # IOU_image_vector.append(max_iou)
        # break


        # print(IOU_dataset_vector)
        # IOU_dataset_vector += IOU_image_vector
        # break




303it [35:59,  7.13s/it]


In [19]:
IOU_dataset_vector

[0,
 0.6548204440470868,
 0.36697748464862406,
 0.6933685688330994,
 0.7680387588298384,
 0.6127588627588627,
 0.7252385865359814,
 0.0003047597938715576,
 0.7408958901698495,
 0.6626438168798777,
 0.24205674283319775,
 0.6371020830735412,
 0.7369217717334406,
 6.248828344685372e-05,
 0.2231300381178529,
 0.24178662442316964,
 0.6828319113905104,
 0.6439461333169729,
 0.04739088189906457,
 0.46654842485008624,
 0.7540482697738963,
 0.8339769493596491,
 0.1846355827792001,
 0.8203670534485892,
 0.7047907380932358,
 0.7972200349207436,
 0.5005495713343592,
 0.8391121362349664,
 0.7638112463005591,
 0.7436311655737531,
 0.7655913283337643,
 0.013401452618335018,
 0.7344119815310597,
 0.690811736406507,
 0.4259724668214454,
 0.001355197181189863,
 0.7735618006442281,
 0.7415006262097233,
 0.823390533570024,
 0.9036922057310288,
 0,
 0.8596660297111306,
 0.6924840910393855,
 0.43135153598189696,
 0.7888557231478747,
 0.4825931183667951,
 0.6083284404498441,
 0.8195015968554231,
 0.857413824

In [20]:
tp = 0
fn = 0
# print(IOU_vector_supervector)
for _, value  in enumerate(IOU_dataset_vector):
    if value > 0.5:
        tp = tp+1
    else:
        fn = fn+1
sensitivity = tp / (tp + fn)

print(sensitivity)

0.717948717948718


Separate the single CSV created to two files, positves

In [21]:
## Update the file name everytime this block is excuted
file_name = 'output-748343c6-bedf-41c0-9696-0ab3a3085ba3.csv'
file_dir = '../dataset/output/features/final/'

# Define the file paths
full_file = os.path.join(file_dir, file_name)
positives_file = os.path.join(file_dir, 'positives.csv')
negatives_file = os.path.join(file_dir, 'negatives.csv')

# Create two CSV files: positives.csv and negatives.csv
with open(positives_file, 'w', newline='') as pfile, open(negatives_file, 'w', newline='') as nfile:
    positive_writer = csv.writer(pfile)
    negative_writer = csv.writer(nfile)

    # Read the original file
    with open(full_file, 'r') as file:
        reader = csv.reader(file)

        # Iterate over each row in the original file
        for row in reader:
            # Check the value of the last column
            if row[-1] == '1.0':
                # Write the row to positives.csv
                positive_writer.writerow(row[:-1])
            elif row[-1] == '0.0':
                # Write the row to negatives.csv
                negative_writer.writerow(row[:-1])

# Print a success message
print("CSV files created successfully.")

CSV files created successfully.


In [13]:
# positive_images_dir = r"../dataset/processed/positive_images"
# groundtruth_path = r'../dataset/processed/groundtruth'

# for idx, file_name in tqdm(enumerate(os.listdir(positive_images_dir))):
    
#     # Read the input image
#     input_image = cv2.imread(
#         os.path.join(
#             positive_images_dir, 
#             file_name
#         ), cv2.IMREAD_UNCHANGED)
    
#     # segment the scales with masses 
#     _, _, _, masses = seg.pipeline(input_image)
    
#     # read the ground truth
#     gt = cv2.imread(os.path.join(groundtruth_path, file_name), cv2.IMREAD_UNCHANGED)
    
#     # get all contours from all the scales and store in a list, this list is to be reset on every image iteration
#     # this is done by iterating over all of the masses, upsampling them, binarize, and find contours
#     scales_contours_vector = []
#     for index, mass_image in enumerate(masses):
#         # upsample
#         mass_image = upsample(mass_image)
        
#         # binarize
#         mass_image = binarize(mass_image)

#         # finding contours
#         contours, _ = cv2.findContours(mass_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
#         scales_contours_vector.append(contours)
        
#         print(mass_image.shape, len(contours))
        
        
        
#     # now upsample and binarize the gt, and get the contours 
#     gt = upsample(gt)
    
#     # binarize just in case
#     gt = binarize(gt)
    
#     # calculate both vector_gt_mask, contours_gt
#     gt_contours, _ = cv2.findContours(gt, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#     print("len: " ,len(gt_contours))
    
#     # Now we can iterate as first block on every contour in gt_contours, and compute the IOU and sensitivity...
#     # We need to integrate feature extraction and mass segmentation from raw image here to generate feature vector...
    
#     # code goes here for calculating the iou for each contour in gt for each contour in segmented
#     # inside: code to extract segmented regions from raw images, save them as 16-bit images to prepare for FE
#     # can also save images with masses ids and iou printed
    
    
#     # print(len(masses))
    
#     break


